# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [4]:
from dotenv import load_dotenv
import os

load_dotenv(".env")

os.environ["OPENAI_BASE_URL"] = "https://openai.vocareum.com/v1"

openai_key = os.getenv("OPENAI_API_KEY", "")


### VectorDB Instance

In [5]:
import chromadb
chroma_client = chromadb.PersistentClient(path="chromadb")
print("✅ Chroma client ready")

✅ Chroma client ready


### Collection

In [6]:
# TODO: Pick one embedding function
# If picking something different than openai, 
# make sure you use the same when loading it
# embedding_fn = embedding_functions.OpenAIEmbeddingFunction()

from lib.voc_embedding import VocareumEmbeddingFunction

embedding_fn = VocareumEmbeddingFunction(model_name="text-embedding-3-small")
print("✅ Using VocareumEmbeddingFunction")


✅ Using VocareumEmbeddingFunction


In [7]:
collection = chroma_client.get_or_create_collection(
        name="udaplay",
        embedding_function=embedding_fn
    )

print("✅ Collection ready:", collection.name)

✅ Collection ready: udaplay


### Add documents

In [8]:
data_dir = "games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )